In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers

import pennylane as qml
import numpy as np

import matplotlib.pyplot as plt

#Loading data

In [5]:
mnist = keras.datasets.mnist

# datasets are numpy.ndarrays
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()     

# normalize the pixels from 0 ~ 225 to 0 ~ 1 
X_train, X_test = X_train / 255.0, X_test / 255.0

11490434/11490434 [==============================] - 2s 0us/step


One hot encoding the labels. We want the labels to be of length cutoff_dim^(num_modes), hence pad with 6 zeros.

In [6]:
def one_hot(labels):  
       
    depth =  4**2                       # 10 classes + 6 zeros for padding
    indices = labels.astype(np.int32)    
    one_hot_labels = np.eye(depth)[indices].astype(np.float32) 
    
    return one_hot_labels

# one-hot encoded labels, each label of length cutoff dimension**2
y_train, y_test = one_hot(Y_train), one_hot(Y_test)

# using only 600 samples for training in this experiment
n_samples = 600
test_samples = 100
X_train, X_test, y_train, y_test = X_train[:n_samples], X_test[:test_samples], y_train[:n_samples], y_test[:test_samples]

In [7]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [8]:
X_test.dtype

dtype('float32')

In [9]:
y_train.dtype

dtype('float32')

#1. Classical circuit

Define classical layers using Keras Sequential. Take in 28x28 image matrices, flatten, and output vectors of length 14. 3 hidden layers with ELU activation.

In [10]:
keras.backend.set_floatx('float64')

model = keras.models.Sequential([
                                 layers.Flatten(input_shape = (28,28)),
                                 layers.Dense(128, activation ="elu"),
                                 layers.Dense(64, activation ="elu"),
                                 layers.Dense(32, activation ="elu"),
                                 layers.Dense(14, activation ="elu") 
                                ])

# More than a million parameters for the classical circuit
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 14)                462       
                                                                 
Total params: 111,278
Trainable params: 111,278
Non-trainable params: 0
_________________________________________________________________


2. Data encoding circuit

Covert the output vectors from the classical circuit into quantum states by using the entries of the vectors as parameters of continous variable gates.

In [11]:
def data_encoding(x):
    qml.Squeezing(x[3], x[4], wires=0)
    qml.Squeezing(x[9], x[10], wires=1)
    
    qml.Beamsplitter(x[5], x[6], wires=[0,1])
    
    qml.Rotation(x[7], wires=0)
    qml.Rotation(x[8], wires=1)
    
    qml.Displacement(x[1], x[2], wires=0)
    qml.Displacement(x[11], x[12], wires=1)
    
    qml.Kerr(x[0], wires=0)
    qml.Kerr(x[13], wires=1)


#3. Quantum neural network circuit

In [12]:
def qnn_layer(v):
    # Interferometer 1
    qml.Beamsplitter(v[0], v[1], wires=[0,1])
    qml.Rotation(v[2], wires=0)
    qml.Rotation(v[3], wires=1)
    
    # Squeezers
    qml.Squeezing(v[4], 0.0, wires=0)
    qml.Squeezing(v[5], 0.0, wires=1)
    
    # Interferometer 2
    qml.Beamsplitter(v[6], v[7], wires=[0,1])
    qml.Rotation(v[8], wires=0)
    qml.Rotation(v[9], wires=1)
    
    # Bias addition
    qml.Displacement(v[10], 0.0, wires=0)
    qml.Displacement(v[11], 0.0, wires=1)
    
    # Non-linear activation function
    qml.Kerr(v[12], wires=0)
    qml.Kerr(v[13], wires=1)

#4. Initialize parameters

In [13]:
def init_weights(layers, modes, active_sd=0.0001, passive_sd=0.1):
    
    # Number of interferometer parameters: beamsplitter + 2 rotations
    M = 2 + 1 + 1  

    int1_weights = tf.random.normal(shape=[layers, M], stddev=passive_sd)
    s_weights = tf.random.normal(shape=[layers, modes], stddev=active_sd)
    int2_weights = tf.random.normal(shape=[layers, M], stddev=passive_sd)
    dr_weights = tf.random.normal(shape=[layers, modes], stddev=active_sd)
    k_weights = tf.random.normal(shape=[layers, modes], stddev=active_sd)

    weights = tf.concat([int1_weights, s_weights, int2_weights, dr_weights, k_weights], axis=1)
    weights = tf.Variable(weights)

    return weights

5. Quantum device

In [14]:
num_modes = 2
cutoff_dim = 4

# select a devide 
dev = qml.device('strawberryfields.fock', wires=num_modes, cutoff_dim=cutoff_dim) 

@qml.qnode(dev, interface="tf")
def quantum_nn(inputs, var):
    # Encode input x into quantum state
    data_encoding(inputs)

    # iterative quantum layers
    for v in var:
        qnn_layer(v)

    return qml.probs(wires=[0, 1])

6. Hybrid circuit

In [15]:
num_layers = 4

# initialize weights for quantum layers
weigths = init_weights(num_layers, num_modes)

# convert the quantum layer to a Keras layer
shape_tup = weigths.shape
weight_shapes = {'var': shape_tup}
qlayer = qml.qnn.KerasLayer(quantum_nn, weight_shapes, output_dim=4)

# add to the classical sequential model
model.add(qlayer)

7. Loss function and optimizer

In [16]:
opt = keras.optimizers.SGD(lr = 0.02)
model.compile(opt, loss = 'categorical_crossentropy', metrics =['accuracy'])

E:\git_repo_asus\CSE499.06(QML)\venv\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.


8. Training

In [ ]:
hybrid = model.fit(X_train.astype('float32'), 
                   y_train.astype('float32'),
                   epochs = 150,
                   batch_size = 64,
                   shuffle = True, 
                   validation_data = (X_test.astype('float32'), y_test.astype('float32')))

10/10 [==============================] - 236s 23s/step - loss: 1.2011 - accuracy: 0.7517 - val_loss: 1.4444 - val_accuracy: 0.5300
Epoch 28/150
10/10 [==============================] - 239s 24s/step - loss: 1.1631 - accuracy: 0.7683 - val_loss: 1.4478 - val_accuracy: 0.6000
Epoch 29/150
10/10 [==============================] - 248s 25s/step - loss: 1.1803 - accuracy: 0.7483 - val_loss: 1.4620 - val_accuracy: 0.6000
Epoch 30/150
10/10 [==============================] - 359s 37s/step - loss: 1.1521 - accuracy: 0.7767 - val_loss: 1.5139 - val_accuracy: 0.5400
Epoch 31/150
10/10 [==============================] - 256s 25s/step - loss: 1.1542 - accuracy: 0.7733 - val_loss: 1.4868 - val_accuracy: 0.5500
Epoch 32/150
10/10 [==============================] - 272s 27s/step - loss: 1.1246 - accuracy: 0.7883 - val_loss: 1.4660 - val_accuracy: 0.5300
Epoch 33/150
10/10 [==============================] - 268s 27s/step - loss: 1.1205 - accuracy: 0.7750 - val_loss: 1.4978 - val_accuracy: 0.5900
Epoch

9. Loss and accuracy graphs

In [ ]:
# ===================================================================================
#                                  Loss History Plot
# ===================================================================================

plt.title('model loss')
plt.plot(hybrid.history['loss'])
plt.ylabel('loss')
plt.show()

In [ ]:
# ===================================================================================
#                                Accuracy History Plot
# ===================================================================================

plt.title('model accuracy')
plt.plot(hybrid.history['accuracy'])
plt.ylabel('accuracy')
plt.show()